In [51]:
import os
import sys 

import pandas as pd
import numpy as np 
import geopandas as gpd
import ee
import geemap
from tqdm import tqdm_notebook

In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
dataset = ee.ImageCollection('WRI/Aqueduct_Flood_Hazard_Maps/V2')

# Get distinct values for the "model" property
distinct_models = dataset.distinct('model')

# Reduce to a list of the "model" property values
model_values = distinct_models.aggregate_array('model').getInfo()

# Print the list of distinct model values
print("Distinct values for 'model':", model_values)

Distinct values for 'model': ['000000000WATCH', '00000NorESM1-M', '0000GFDL-ESM2M', '0000HadGEM2-ES', '00IPSL-CM5A-LR', 'MIROC-ESM-CHEM']


In [52]:
poly = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_2.json')
poly = poly.to_crs(epsg=4326)
gid_2 = poly['GID_2'].to_list()
flood_depth = []

# Define the dataset collection
# https://developers.google.com/earth-engine/datasets/catalog/WRI_Aqueduct_Flood_Hazard_Maps_V2#image-properties
# https://files.wri.org/d8/s3fs-public/aqueduct-floods-methodology.pdf?_gl=1*1u9ay0w*_gcl_au*MTg3NzAwNTI0OC4xNzI0MjQwNTIw
# https://www.usgs.gov/centers/new-jersey-water-science-center/floods-recurrence-intervals-and-100-year-floods 
flood = ee.ImageCollection('WRI/Aqueduct_Flood_Hazard_Maps/V2').filter(ee.Filter.eq('climatescenario', 'rcp8p5')) \
                                                         .filter(ee.Filter.eq('floodtype', 'inunriver')) \
                                                         .filter(ee.Filter.eq('returnperiod', 100))\
                                                         .filter(ee.Filter.eq('model', '0000GFDL-ESM2M'))\
                                                         .filter(ee.Filter.eq('year', 2030))\
                                                         .sum()\
                                                         .select('inundation_depth')

for i in tqdm_notebook(range(len(poly))):

    r = poly.iloc[i:i+1]
    region = geemap.gdf_to_ee(r).geometry()

    stats = flood.reduceRegion(
                      geometry=region,
                      reducer=ee.Reducer.mean(),
                      scale=1000,  
                      crs='EPSG:4326', 
                      tileScale=14,
                      bestEffort=True
                )
    
    
    try:
        depth = stats.getInfo()['inundation_depth']
        flood_depth.append(depth)
        
    except Exception as e:
         print(e)
         continue
    
flood_df = pd.DataFrame({'GID_2': gid_2,
                         'inundation_depth': flood_depth})
flood_df['year'] = 2022 
    

C:\Users\sonle\AppData\Local\Temp\ipykernel_1736\3592506215.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(poly))):


  0%|          | 0/161 [00:00<?, ?it/s]

In [59]:
poly = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_0.json')
region = geemap.gdf_to_ee(poly).geometry()

# Define the dataset collection
for year in [2030, 2050, 2080]:
    flood = ee.ImageCollection('WRI/Aqueduct_Flood_Hazard_Maps/V2').filter(ee.Filter.eq('climatescenario', 'rcp8p5')) \
                                                            .filter(ee.Filter.eq('floodtype', 'inunriver')) \
                                                            .filter(ee.Filter.eq('returnperiod', 100))\
                                                            .filter(ee.Filter.eq('model', '0000GFDL-ESM2M'))\
                                                            .filter(ee.Filter.eq('year', year))\
                                                            .sum()\
                                                            .select('inundation_depth')


    task = ee.batch.Export.image.toDrive(
        image=flood,
        description=f'SRB_{year}_1in100_rcp8p5',
        folder='wri_flooding',
        region=region,
        scale=1000,
        crs='EPSG:4326',
        fileNamePrefix=f'SRB_{year}_1in100_rcp8p5',
        # maxPixels=1e13,
        fileFormat='GeoTIFF'
    )
    task.start()

In [61]:
from datetime import datetime

from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

In [62]:
def query_osm_overpass(area_name, key, value):

    nominatim = Nominatim()
    areaId = nominatim.query(area_name).areaId()
    overpass = Overpass()
    query = overpassQueryBuilder(area=areaId, elementType='way', selector=f'"{key}"="{value}"', out='body', includeGeometry=True)
    result = overpass.query(query, timeout=120)
    
    return result



def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)


def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_{service_type}_tiles.zip"
    return url

def merge_with_connectivity(out, ookla):
    out = out[~out.lat.isna()]
    out['quadkey'] = out.apply(make_quadkey, axis=1)
    out['quadkey'] = out['quadkey'].astype('str')
    connectivity = ookla[ookla.quadkey.isin(out['quadkey'])]
    connectivity = connectivity.set_index('quadkey')
    tmp = out.set_index('quadkey')
    tmp.index = tmp.index.astype('str')
    tmp = tmp.join(connectivity, how='left')
    tmp = tmp.reset_index().drop_duplicates(
        subset=['name', 'quadkey'],
        keep='last').reset_index(drop=True)
    # return connectivity
    return tmp

def make_quadkey(row):
    return qk.from_geo((row['lat'], row['lon']), 16)

def make_details(element):
    tags = element.tags()
    if 'name' in tags:
        out = dict()
        out['amenity'] = tags['amenity']
        out['name'] = tags['name']
        if 'lat' in tags:
            out['lat'] = tags['lat']
            out['lon'] = tags['lon']
        elif bool(element.centerLat()):
            out['lat'] = element.centerLat()
            out['lon'] = element.centerLon()
        elif bool(element.geometry()):
            out['lat'] = element.geometry()['coordinates'][0][0][1]
            out['lon'] = element.geometry()['coordinates'][0][0][0]
        else:
            out['lat'] = None
            out['lon'] = None
        return out
    else:
        return None
    
def filter_elements(result):
    elements = list()
    for element in result.elements():
        elements.append(make_details(element))
    filtered = [element for element in elements if element is not None]
    return filtered

In [63]:
result = query_osm_overpass('Serbia', 'highway', 'motorway')

[overpass] downloading data: [timeout:120][out:json];area(3609088937)->.searchArea;(way["highway"="motorway"](area.searchArea);); out body geom;


In [68]:
result.elements()[10].tags()

{'highway': 'motorway',
 'int_ref': 'E 70',
 'lanes': '3',
 'lit': 'yes',
 'maxspeed': '40',
 'note': 'ZA POČETNIKE: Naplatna rampa je mapirana ispravno, sa svim propratnim podacima: toll, lanes, maxspeed, itd. Nemojte docrtavati puteve kao posebne trake. Nije ispravno.',
 'oneway': 'yes',
 'placement': 'right_of:1',
 'ref': 'A3',
 'surface': 'asphalt'}

In [70]:
roads = gpd.read_file("D:\Work\WB\LDT\countries\SRB\\raw_data\serbia-230101-free.shp\gis_osm_roads_free_1.shp")
roads

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,4432186,5122,residential,Fő utca,None,B,0,0,F,F,"LINESTRING (20.01588 46.17707, 20.01869 46.178..."
1,4759407,5122,residential,None,None,B,0,0,F,F,"LINESTRING (22.65914 42.87117, 22.65911 42.871..."
2,4759488,5122,residential,None,None,B,0,0,F,F,"LINESTRING (22.6802 42.86464, 22.68017 42.8646..."
3,4907997,5113,primary,Саве Ковачевића,12,B,50,0,F,F,"LINESTRING (20.4841 45.42493, 20.4831 45.42412..."
4,4907998,5114,secondary,Бирчанинова,None,B,0,0,F,F,"LINESTRING (20.40807 45.37685, 20.40849 45.37722)"
...,...,...,...,...,...,...,...,...,...,...,...
453627,1126353054,5115,tertiary,Новосадског Сајма,None,F,50,0,F,F,"LINESTRING (19.83059 45.25472, 19.83073 45.254..."
453628,1126353055,5122,residential,Боре Продановића,None,B,0,0,F,F,"LINESTRING (19.83059 45.25472, 19.83062 45.254..."
453629,1126353056,5115,tertiary,Новосадског Сајма,None,F,50,0,F,F,"LINESTRING (19.83058 45.25477, 19.83045 45.25476)"
453630,1126353057,5115,tertiary,Новосадског Сајма,None,F,50,0,F,F,"LINESTRING (19.83045 45.25476, 19.8304 45.2547..."
